# **TuriCreate Recommender Model**
The following notebook includes an example of data import, model training and validation evaluation.

In [1]:
import turicreate as tc
import pandas as pd
import time
#from skafossdk import *
import common.save_models as sm

## **The Data**
There are 2 main data inputs for a recommender model:
- **items**: items that we want to recommend to a given user, ex. apples
- **actions**: actions users have taken on items, ex. John bought apples

In our example here, we have movies (items) and ratings by users on those movies(actions).

In [2]:
%%capture 
actions = tc.SFrame.read_csv('https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/ratings.csv'); 
items = tc.SFrame.read_csv('https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/movies.csv');

In [3]:
pd_actions = actions.to_dataframe()
sample_user_ids = pd.Series(pd_actions['userId'].unique()).sample(10000)
actions = tc.SFrame(pd_actions[pd_actions['userId'].isin(sample_user_ids)])

In [4]:
%%capture 
# split the training and validation sets up
training_data, validation_data = tc.recommender.util.random_split_by_user(actions, 'userId', 'movieId')

In [5]:
%%capture 
# build the recommender
model = tc.recommender.create(training_data, 'userId', 'movieId')

In [6]:
%%capture
# grab the results of the model
results = model.recommend();

In [7]:
# print the validation data
validation_data.print_rows(num_rows=10)

+--------+---------+--------+------------+
| userId | movieId | rating | timestamp  |
+--------+---------+--------+------------+
|  197   |    50   |  4.5   | 1360869306 |
|  311   |    15   |  1.0   | 982512302  |
|  311   |    50   |  5.0   | 982512890  |
|  311   |   160   |  1.0   | 982512369  |
|  311   |   204   |  3.0   | 982511941  |
|  311   |   327   |  1.0   | 982512444  |
|  311   |   377   |  4.0   | 982511341  |
|  311   |   379   |  2.0   | 982512038  |
|  311   |   442   |  3.0   | 982512204  |
|  311   |   464   |  3.0   | 982511777  |
+--------+---------+--------+------------+
[31446 rows x 4 columns]



In [8]:
# evaluate the model
model.evaluate(validation_data)



Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |  0.108108108108108  | 0.002364968897897846 |
|   2    | 0.10360360360360361 | 0.005659827258681899 |
|   3    | 0.10443777110443779 | 0.009543169542703336 |
|   4    | 0.10435435435435433 | 0.014361621897442915 |
|   5    | 0.10450450450450448 | 0.018888683650579035 |
|   6    | 0.10927594260927605 | 0.023959281802789098 |
|   7    | 0.11297011297011299 | 0.029681887378519574 |
|   8    | 0.11774274274274286 | 0.03586178940005085  |
|   9    | 0.11911911911911911 | 0.04163323765320732  |
|   10   | 0.12042042042042055 | 0.04712799440584593  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]



{'precision_recall_by_user': Columns:
 	userId	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 17982
 
 Data:
 +--------+--------+-----------+--------+-------+
 | userId | cutoff | precision | recall | count |
 +--------+--------+-----------+--------+-------+
 |  197   |   1    |    0.0    |  0.0   |   1   |
 |  197   |   2    |    0.0    |  0.0   |   1   |
 |  197   |   3    |    0.0    |  0.0   |   1   |
 |  197   |   4    |    0.0    |  0.0   |   1   |
 |  197   |   5    |    0.0    |  0.0   |   1   |
 |  197   |   6    |    0.0    |  0.0   |   1   |
 |  197   |   7    |    0.0    |  0.0   |   1   |
 |  197   |   8    |    0.0    |  0.0   |   1   |
 |  197   |   9    |    0.0    |  0.0   |   1   |
 |  197   |   10   |    0.0    |  0.0   |   1   |
 +--------+--------+-----------+--------+-------+
 [17982 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'precisio

In [9]:
# export to coreml
coreml_model_name = "recommender.mlmodel"
res = model.export_coreml(coreml_model_name)

# compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

This model is exported as a custom Core ML model. In order to use it in your
application, you must also include "libRecommender.dylib". For additional
details see:
https://apple.github.io/turicreate/docs/userguide/recommender/coreml-deployment.html


In [ ]:
# save to Skafos
sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
                     compressed_model = compressed_model,
                     permissions = 'public')